In [1]:
import csv
import os
import pandas as pd
import platform
import selenium
import pickle
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
dicionario_musicas = {'pagina': [],
                     'indice_musica': [],
                     'titulo_musica': [],
                     'nome_artista': [],
                     'letra': [],
                     'album': [],
                     'data_lancamento': [],
                     'tags': []}

musicas_df = pd.DataFrame.from_dict(dicionario_musicas)
nome_arquivo_musicas = 'musicas.csv'
if os.path.isfile(nome_arquivo_musicas):
    musicas_df = pd.read_csv(nome_arquivo_musicas)
else:
    musicas_df.to_csv(nome_arquivo_musicas, index=False)

indice_inicial = 50
paginas = musicas_df['pagina'].unique()
if paginas.size > 0:
    indice_inicial = paginas[-1]

print(indice_inicial)

32


In [12]:
sistema_operacional = platform.system()

if sistema_operacional == 'Linux':
    caminho_webdriver = os.path.join('chromedriver_linux64', 'chromedriver')
elif sistema_operacional == 'Windows':
    caminho_webdriver = os.path.join('chromedriver_win32', 'chromedriver.exe')

servico = Service(executable_path=caminho_webdriver)

In [ ]:
opcoes = webdriver.ChromeOptions()
#opcoes.add_argument("--headless=new")
navegador = webdriver.Chrome(service=servico, options = opcoes)
espera_navegador = WebDriverWait(navegador, 20)

for i in range(indice_inicial, 0, -1):
    link = f'https://genius.com/tags/mpb/all?page={i}'
    navegador.get(link)
    
    while True:
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        if not (len(navegador.find_elements(By.XPATH, "//a[contains(text(), 'more')]"))) > 0:
            break
    
    elementos_musica_id = '//span[@class="title_with_artists"]'
    elementos_musica = espera_navegador.until(EC.presence_of_all_elements_located((By.XPATH, elementos_musica_id)))
    
    for j in range(len(elementos_musica)):
        musica = elementos_musica[j]
        pai = musica.find_element(By.XPATH, '..')
        titulo_musica = musica.find_element(By.CLASS_NAME, 'song_title').get_attribute("innerText")
        
        elemento_artistas = musica.find_element(By.CLASS_NAME, 'artist_name')
        nome_artista = elemento_artistas.find_element(By.CLASS_NAME, 'primary_artist_name').get_attribute("innerText")
        
        lista_artistas_destacados = []
        try:
            nomes_artistas_destacados = elemento_artistas.find_elements(By.CLASS_NAME, 'featured_artist_name')
            for nome_artista_destacado in nomes_artistas_destacados:
                lista_artistas_destacados.append(nome_artista_destacado.get_attribute("innerText"))
        except Exception as e:
            print(f'Erro de artistas destacados: {str(e)}')

        link_letra = pai.get_attribute('href')
        navegador.execute_script(f'window.open("{link_letra}","_blank");')
        navegador.switch_to.window(navegador.window_handles[1])
        
        letra = ''
        try:
            identificador_containers_letra_id = '//div[@data-lyrics-container="true"]'
            containers_letra = espera_navegador.until(EC.presence_of_all_elements_located((By.XPATH, identificador_containers_letra_id)))
            for container in containers_letra:
                letra_atual = container.get_attribute("innerText")
                letra += letra_atual
        except Exception as e:
            print(f'Erro na letra: {str(e)}')

        album = ''
        try:
            album = navegador.find_element(By.XPATH, "//a[contains(@class, 'PrimaryAlbum')]").get_attribute("innerText")
        except Exception as e:
            print(f'Erro no nome do álbum: {str(e)}')
        
        data_lancamento = ''
        try:
            identificador_data_lancamento = "//span[contains(@class, 'MetadataStats')]"
            elementos_data_lancamento = navegador.find_elements(By.XPATH, identificador_data_lancamento)
            data_lancamento = elementos_data_lancamento[0].get_attribute("innerText")
        except Exception as e:
            print(f'Erro na data de lançamento: {str(e)}')
        
        tags = ''
        try:
            identificador_tags = "//a[contains(@class, 'SongTags__Tag')]"
            elementos_tags = navegador.find_elements(By.XPATH, identificador_tags)
            tags = ','.join([tag.get_attribute("innerText") for tag in elementos_tags])
        except Exception as e:
            print(f'Erro nas tags: {str(e)}')
        
        nova_linha = [i, j, titulo_musica, nome_artista, letra, album, data_lancamento, tags]
        df_nova_linha = pd.DataFrame([nova_linha])
        df_nova_linha.to_csv(nome_arquivo_musicas, mode='a', header=False, index=False)
        
        navegador.close()
        navegador.switch_to.window(navegador.window_handles[0])

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro na letra: Message: 

Erro na letra: Message: 

Erro no nome do álbum: Message: no such element: Unable to locat

...

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003BA813+48355]
	(No symbol) [0x0034C4B1]
	(No symbol) [0x00255358]
	(No symbol) [0x002809A5]
	(No symbol) [0x00280B3B]
	(No symbol) [0x002AE232]
	(No symbol) [0x0029A784]
	(No symbol) [0x002AC922]
	(No symbol) [0x0029A536]
	(No symbol) [0x002782DC]
	(No symbol) [0x002793DD]
	GetHandleVerifier [0x0061AABD+2539405]
	GetHandleVerifier [0x0065A78F+2800735]
	GetHandleVerifier [0x0065456C+2775612]
	GetHandleVerifier [0x004451E0+616112]
	(No symbol) [0x00355F8C]
	(No symbol) [0x00352328]
	(No symbol) [0x0035240B]
	(No symbol) [0x00344FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

In [ ]:
m_df = pd.read_csv('musi')